# Get 16S correlations 
- calculate corr tables used for 16S & guilds
- CH4 ~ genus, OTU (genus preferred for tables/figs) 
- This is slow (very slow for OTUs) - correlation cells commented out
- ...
- note an older method exisited for separate OTU & genus corrs, appended at end 
- not sure what bug was, recreated sloppy code patch


# 1) import data

- post processing here, should be in a different module...but isn't

In [30]:
suppressMessages(library(dplyr))
suppressMessages(library(rlang))
library(mctoolsr)

You're using mctoolsr (v.0.1.1.2). Direct inquiries to:
'https://github.com/leffj/mctoolsr'



In [1]:
# Import OTU Table
SilvaOTUs <- read.table("../silvaOTUs/Silva_OTU_VSTcpm.txt", sep='\t')#, row.names=1)
#iTaggerOTUs <- read.table("../data/iTaggerOTUs/SF_Sal_OTU_VSTcpm.txt", sep='\t', row.names=1, header=T)
row.names(SilvaOTUs) <- SilvaOTUs[,"OTU"]                                # Make OTU row names -- required!!!  only for silva?
SilvaOTUs <- SilvaOTUs[,-1]

In [18]:
# SilvaOTUs

In [3]:
# otu_V <- iTaggerOTUs 
otu_V <- SilvaOTUs

In [4]:
# Sort OTU table                                                                      
otu_V <- otu_V[order(otu_V$Consensus.lineage),]                                                         # sort by lineage

# Make new top level plotting var (should be in PRE-PROCESS ? )
otu_V$Taxonomy <- ifelse(otu_V$Phylum == "Proteobacteria", paste(otu_V$Class), paste(otu_V$Phylum))    # head(otu_V)
otu_V <- data.frame(otu_V)

In [5]:
# METADATA
metaDB <-read.table("../data/meta/SF_sal_meta_FIX3.txt", sep="\t", header=TRUE)               # import Mapping    # # try keeping all params...


In [6]:
# Prune metadata to only iTag samples
# Get Sample names in OTU table               
OTU_samps <- data.frame('Sample'=colnames(otu_V))                                        # OTU_samps

# Merge site order and Samples
Meta_iTag <- merge(metaDB, OTU_samps, by='Sample')                                       #colnames(metaDB)
rownames(Meta_iTag) <- Meta_iTag$Sample

In [7]:
# add any extra variables as needed                                                      # head(Meta_iTag)  #colnames(Meta_iTag)
CH4 <- Meta_iTag$CH4_ug_m2_h
Meta_iTag$CH4_logn1 <- log10(CH4 - (1.05*min(CH4)))   

In [8]:
#metaDB
# Meta_iTag['CH4_logn1']

## 1a) log2 transform count data
- equivalent code somewhere?

In [9]:
# otu_V

# 2) Corr ranks to OTU & genus level
- OTUs are a separate call as slow

In [10]:
source("../modules/6_Corr_ranks_module_v0.3.3_strip.R")

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units



Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess




In [11]:
# Only to genus level: --- OOPS THIS IS OTU somehow, bugs.  Genus should still be valid via aggregation?
CH4corrRanks <- corr_TaxRanks(otu_V, Meta_iTag, "CH4_logn1")                                  
head(CH4corrRanks)

,Consensus.lineage,Kingdom,Phylum,Class,Order,Family,Genus,Taxonomy,OTU,Kingdom_r,Phylum_r,Class_r,Order_r,Family_r,Genus_r,Taxonomy_r,OTU_r
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
684,k__Archaea;p__;c__;o__;f__;g__,Archaea,,PH,PH,PH,PH,,otu_1614,0.4277461,0.0000000,-0.5364815,-0.5364815,-0.5364815,-0.5364815,0.0000000,-0.3116288
2938,k__Archaea;p__;c__;o__;f__;g__,Archaea,,PH,PH,PH,PH,,otu_3660,0.4277461,0.0000000,-0.5364815,-0.5364815,-0.5364815,-0.5364815,0.0000000,-0.2996046
3770,k__Archaea;p__;c__;o__;f__;g__,Archaea,,PH,PH,PH,PH,,otu_4420,0.4277461,0.0000000,-0.5364815,-0.5364815,-0.5364815,-0.5364815,0.0000000,0.0000000
4198,k__Archaea;p__Aenigmarchaeota;c__Aenigmarchaeia;o__Aenigmarchaeales;f__;g__,Archaea,Aenigmarchaeota,Aenigmarchaeia,Aenigmarchaeales,AenigmarchaealesOR,AenigmarchaealesOR,Aenigmarchaeota,otu_4814,0.4277461,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
1013,k__Archaea;p__Altiarchaeota;c__Altiarchaeia;o__Altiarchaeales;f__Altiarchaeaceae;g__Cand. Altiarchaeum,Archaea,Altiarchaeota,Altiarchaeia,Altiarchaeales,Altiarchaeaceae,Cand. Altiarchaeum,Altiarchaeota,otu_1912,0.4277461,0.6131857,0.6131857,0.6131857,0.6131857,0.0000000,0.6131857,0.5978474
5439,k__Archaea;p__Altiarchaeota;c__Altiarchaeia;o__Altiarchaeales;f__Altiarchaeaceae;g__Cand. Altiarchaeum,Archaea,Altiarchaeota,Altiarchaeia,Altiarchaeales,Altiarchaeaceae,Cand. Altiarchaeum,Altiarchaeota,otu_5947,0.4277461,0.6131857,0.6131857,0.6131857,0.6131857,0.0000000,0.6131857,0.5190304


In [31]:
# CH4corrRanks

In [32]:
write.table(CH4corrRanks, "correlations/CH4corrRanks_SilvaOTU.txt", sep = '\t')

### b) Genus only
- sloppy, forgot how to R -- this should really use subset by numeric cols
- here grab column lists, need to reorder outplut

In [33]:
# Get columns for taxa
otu_cols <- colnames(otu_V)
tax_cols <- unlist(list("Consensus.lineage", "Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Taxonomy"))

# taxa only data to append to aggregated
tax_data = otu_V[tax_cols]

# aggregate counts by genus 
otu_V_Gagg <- agg_by_cat(otu_V, "Genus")
otu_V_G <- unique(merge(otu_V_Gagg, tax_data))

# reorder columns as initial
otu_V_G <- otu_V_G[otu_cols]

#### GET CORRS - not esp fast

In [34]:
# GET CORRS
CH4corrRanks_genus <- corr_TaxRanks(otu_V_G, Meta_iTag, "CH4_logn1")                                  # head(CH4corrRanks)

In [35]:
# Drop extra columns
CH4corrRanks_genus$OTU <-NULL
CH4corrRanks_genus$OTU_r <-NULL

In [36]:
# CH4corrRanks_genus
head(CH4corrRanks_genus)

,Consensus.lineage,Kingdom,Phylum,Class,Order,Family,Genus,Taxonomy,Kingdom_r,Phylum_r,Class_r,Order_r,Family_r,Genus_r,Taxonomy_r
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
663,k__Archaea;p__;c__;o__;f__;g__,Archaea,,PH,PH,PH,PH,,0.2746068,0.0000000,-0.5364815,-0.5364815,-0.5364815,-0.5364815,0.0000000
407,k__Archaea;p__Aenigmarchaeota;c__Aenigmarchaeia;o__Aenigmarchaeales;f__;g__,Archaea,Aenigmarchaeota,Aenigmarchaeia,Aenigmarchaeales,AenigmarchaealesOR,AenigmarchaealesOR,Aenigmarchaeota,0.2746068,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
45,k__Archaea;p__Altiarchaeota;c__Altiarchaeia;o__Altiarchaeales;f__Altiarchaeaceae;g__Cand. Altiarchaeum,Archaea,Altiarchaeota,Altiarchaeia,Altiarchaeales,Altiarchaeaceae,Cand. Altiarchaeum,Altiarchaeota,0.2746068,0.6131857,0.6131857,0.6131857,0.6131857,0.0000000,0.6131857
478,k__Archaea;p__Asgardarchaeota;c__Lokiarchaeia;o__;f__;g__,Archaea,Asgardarchaeota,Lokiarchaeia,LokiarchaeiaCL,LokiarchaeiaCL,LokiarchaeiaCL,Asgardarchaeota,0.2746068,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
152,k__Archaea;p__Crenarchaeota;c__;o__;f__;g__,Archaea,Crenarchaeota,CrenarchaeotaPH,CrenarchaeotaPH,CrenarchaeotaPH,CrenarchaeotaPH,Crenarchaeota,0.2746068,0.1863573,-0.3348913,-0.3348913,-0.3348913,-0.3348913,0.1863573
1053,k__Archaea;p__Crenarchaeota;c__Bathyarchaeia;o__;f__;g__,Archaea,Crenarchaeota,Bathyarchaeia,BathyarchaeiaCL,BathyarchaeiaCL,BathyarchaeiaCL,Crenarchaeota,0.2746068,0.1863573,0.1676165,0.1676165,0.1676165,0.1676165,0.1863573


In [37]:
write.table(CH4corrRanks_genus, "correlations/CH4corrRanks_Silva_genus.txt", sep = '\t')

# 3) Delta Only

In [38]:
# Meta_iTag

In [39]:
# Separate Delta Sites (oligo and FW)
# levels(Meta_iTag$SALTgroup)
Meta_iTag_FW <- Meta_iTag[Meta_iTag$SALTgroup =="FW",]
Meta_iTag_Oligo <- Meta_iTag[Meta_iTag$SALTgroup =="Oligo",]

Meta_iTag_Delta <-rbind(Meta_iTag_FW, Meta_iTag_Oligo)                  # Meta_iTag_Delta

Delta_sites <-Meta_iTag_Delta[,"Sample"]                                # Delta sites

In [40]:
# Get taxonomy columns, combine with Delta sites
tax_cols <- unlist(list("Consensus.lineage", "Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Taxonomy"))
Delta_tax <- append(Delta_sites, tax_cols)

In [41]:
otu_V_Delta <- otu_V[Delta_tax]
# otu_V_Delta

### a) Get correlations by OTU

In [42]:
CH4corrRanks_Delta <- corr_TaxRanks(otu_V_Delta, Meta_iTag_Delta, "CH4_logn1")                                  
head(CH4corrRanks)

,Consensus.lineage,Kingdom,Phylum,Class,Order,Family,Genus,Taxonomy,OTU,Kingdom_r,Phylum_r,Class_r,Order_r,Family_r,Genus_r,Taxonomy_r,OTU_r
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
684,k__Archaea;p__;c__;o__;f__;g__,Archaea,,PH,PH,PH,PH,,otu_1614,0.4277461,0.0000000,-0.5364815,-0.5364815,-0.5364815,-0.5364815,0.0000000,-0.3116288
2938,k__Archaea;p__;c__;o__;f__;g__,Archaea,,PH,PH,PH,PH,,otu_3660,0.4277461,0.0000000,-0.5364815,-0.5364815,-0.5364815,-0.5364815,0.0000000,-0.2996046
3770,k__Archaea;p__;c__;o__;f__;g__,Archaea,,PH,PH,PH,PH,,otu_4420,0.4277461,0.0000000,-0.5364815,-0.5364815,-0.5364815,-0.5364815,0.0000000,0.0000000
4198,k__Archaea;p__Aenigmarchaeota;c__Aenigmarchaeia;o__Aenigmarchaeales;f__;g__,Archaea,Aenigmarchaeota,Aenigmarchaeia,Aenigmarchaeales,AenigmarchaealesOR,AenigmarchaealesOR,Aenigmarchaeota,otu_4814,0.4277461,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
1013,k__Archaea;p__Altiarchaeota;c__Altiarchaeia;o__Altiarchaeales;f__Altiarchaeaceae;g__Cand. Altiarchaeum,Archaea,Altiarchaeota,Altiarchaeia,Altiarchaeales,Altiarchaeaceae,Cand. Altiarchaeum,Altiarchaeota,otu_1912,0.4277461,0.6131857,0.6131857,0.6131857,0.6131857,0.0000000,0.6131857,0.5978474
5439,k__Archaea;p__Altiarchaeota;c__Altiarchaeia;o__Altiarchaeales;f__Altiarchaeaceae;g__Cand. Altiarchaeum,Archaea,Altiarchaeota,Altiarchaeia,Altiarchaeales,Altiarchaeaceae,Cand. Altiarchaeum,Altiarchaeota,otu_5947,0.4277461,0.6131857,0.6131857,0.6131857,0.6131857,0.0000000,0.6131857,0.5190304


In [43]:
# CH4corrRanks_Delta

In [44]:
write.table(CH4corrRanks_Delta, "correlations/CH4corrRanks_Silva_Delta_OTU.txt", sep = '\t')

### b) Genus only
- sloppy, forgot how to R.

In [45]:
# taxa only data
# tax_data = otu_V[tax_cols]

In [46]:
# aggregate counts 
otu_V_Delta_Gagg <- agg_by_cat(otu_V_Delta, "Genus")
otu_V_Delta_G <- unique(merge(otu_V_Delta_Gagg, tax_data))

In [47]:
# otu_V_Delta_G

In [48]:
# Reorder data
otu_V_Delta_G <- otu_V_Delta_G[Delta_tax]

In [49]:
# GET CORRS
CH4corrRanks_Delta_genus <- corr_TaxRanks(otu_V_Delta_G, Meta_iTag_Delta, "CH4_logn1")                                  # head(CH4corrRanks)

In [50]:
# Drop extra columns
CH4corrRanks_Delta_genus$OTU <-NULL
CH4corrRanks_Delta_genus$OTU_r <-NULL

In [32]:
# CH4corrRanks_Delta_genus

In [51]:
write.table(CH4corrRanks_Delta_genus, "correlations/CH4corrRanks_Silva_Delta_genus.txt", sep = '\t')

## 4) Older method 
- cleaner but didn't work?   
- possibly OTU formatting problem

In [34]:
## A) Recalculate corr METHODS for controller, using functions from correlations module:
# source("Corr_ranks_module_v0.3.2_strip.R")

# Only to genus level:
# CH4corrRanks <- corr_TaxRanks(otu_V, Meta_iTag, "CH4_logn1")                                  # head(CH4corrRanks)

# Append corrs at OTU level --  SLOW (10 min), need only for iTol? 
# OTU_ra <-append_OTU_corr_v(otu_V, CH4corr_ranks, "CH4_logn1")            

# For Delta samples subset
# CH4corrRanks_Delta_OTU <-append_OTU_corr_v(Delta_OTU, CH4corrRanks_Delta, "CH4_logn1")   
# head(CH4corrRanks_Delta_OTU)

# typically saved to file
# write.table(OTU_ra, "CH4_TaxRankOTU_corr_tab_all_sites.txt", sep="\t", col.names = TRUE, row.names = FALSE, quote=FALSE)
# write.table(CH4corrRanks_Delta_OTU, "CH4_TaxRankOTU_corr_tab_Delta_sites.txt", sep="\t", col.names = TRUE, row.names = FALSE, quote=FALSE)

## B) Read EXISTING corr table: 
# CH4corrRanksOTU <-read.table("CH4_TaxRankOTU_corr_tab_all_sites.txt", sep="\t", header=T)      #head(CH4corrRanksOTU)

## C) Read EXISTING Delta corr table: 
# CH4corrRanksOTU_Delta <-read.table("CH4_TaxRankOTU_corr_tab_Delta_sites.txt", sep="\t", header=T)  #head(CH4corrRanksOTU_Delta)
